In [1]:
import pandas as pd
import numpy as np
import re,glob,os,secrets
import urllib.parse
import urllib.request
from multiprocessing.pool import ThreadPool
import urllib
import pandas as pd
import operator
import functools
from operator import methodcaller

URL = 'https://www.uniprot.org/uploadlists/'

def create_chunks(list_name, n):
    for i in range(0, len(list_name), n):
        yield list_name[i:i + n]
        
def mapping_ids(ids2map, source_fmt='P_GI',target_fmt='ACC', output_fmt='tab'):
    if hasattr(ids2map, 'pop'):
            ids2map = ' '.join(ids2map)
    params = {
    'from': source_fmt,
    'to': target_fmt,
    'format': output_fmt,
    'query': ids2map
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(URL, data)
    with urllib.request.urlopen(req) as f:
        response = f.read().decode('utf-8')
    return response

def concat_dfs(path,extension):
    extension = extension
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    combined_dfs = pd.concat([pd.read_table(f) for f in all_filenames ])
    
    return combined_dfs

def taxonomy_metag(basta_output, besthits):
    columns_basta_output = ['id_read','taxonomy']
    basta_output = pd.DataFrame(np.vstack([basta_output.columns,basta_output]))
    basta_output.columns = columns_basta_output
    taxonomy = basta_output["taxonomy"].str.split(";",expand=True)
    basta_output["taxonomy"] = taxonomy[5]
    besthits = pd.merge(besthits,basta_output,left_on="qseqid",right_on="id_read",how="left")
    besthits = besthits.drop("id_read",1)
    besthits = besthits.loc[besthits["taxonomy"]!="unknown"]
    besthits = besthits.loc[besthits["taxonomy"].notna()]
    besthits = besthits.reset_index(drop=True)
    besthits = besthits[~besthits.duplicated(['KO','taxonomy'])].reset_index(drop=True)
    return besthits

In [22]:
df_besthits = pd.read_table("C:/Users/eulle/Documents/metagenomica/marbella/besthits_E7.m8")

columns_csv = ['qseqid' ,'sseqid' ,'pident' ,'length','mismatch','gapopen','qstart' ,'qend' ,'sstart' ,'send' ,'evalue' ,'bitscore']
row_values = pd.Series(df_besthits.columns)
df_besthits = pd.DataFrame(np.vstack([df_besthits.columns,df_besthits]))
df_besthits.columns = columns_csv


In [23]:
gi_number = [re.findall(r"(?<=GI:|gi:).*?(?=\||p)", ii[1]['sseqid']) for ii in df_besthits.iterrows()]
gi_number = functools.reduce(operator.iconcat, gi_number, [])
gi_number = list(set(gi_number))

uniprot_id = [re.findall(r"(?<=BL:|ot:).*?(?=\||p)",ii[1]['sseqid']) for ii in df_besthits.iterrows()]
uniprot_id = functools.reduce(operator.iconcat, uniprot_id, [])
uniprot_id = list(set(uniprot_id))

chunk_gi = list(create_chunks(gi_number,1000))
chunk_uniprot = list(create_chunks(uniprot_id,1000))

In [24]:
# Criar uma função para escrever os arquivos
map_gi = ThreadPool(10).imap(mapping_ids,chunk_gi)
map_uniprot = ThreadPool(10).imap(functools.partial(mapping_ids,source_fmt='ACC',target_fmt='KO_ID'),chunk_uniprot)

parent_dir = "D:/eulle/importante/"
dir_gi_uni = "gi_number-uni"
dir_uni_ko = "uniprot-ko_id"

if(os.path.isdir('D:/eulle/importante/gi_number-uni/')==False):
    os.mkdir(os.path.join(parent_dir, dir_gi_uni))
    for index,ii in enumerate(map_gi):
        with open('D:/eulle/importante/gi_number-uni/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
else:
    for index,ii in enumerate(map_gi):
        with open('D:/eulle/importante/gi_number-uni/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
            
            
if(os.path.isdir('D:/eulle/importante/uniprot-ko_id/')==False):        
    os.mkdir(os.path.join(parent_dir, dir_uni_ko))
    for index,ii in enumerate(map_uniprot): 
        with open('D:/eulle/importante/uniprot-ko_id/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
else:
    for index,ii in enumerate(map_gi):
        with open('D:/eulle/importante/uniprot-ko_id/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
    

In [25]:
# Pensar em uma forma de melhorar esses downloads
gi_uniprot_concat = concat_dfs(os.chdir("D:/eulle/importante/gi_number-uni/"),"csv")
gi_uniprot_concat = gi_uniprot_concat.drop_duplicates("From")
gi_uniprot_concat = gi_uniprot_concat.reset_index(drop=True)
gi_uniprot_concat.columns = ["GI","Uniprot_ID"]

gi_uniprot_concat.to_csv("D:/eulle/importante/df_gi_uniprot-E7.csv",sep="\t",encoding="utf-8",index=False)

list_gi_uniprot = list(gi_uniprot_concat["Uniprot_ID"])
chunk_uniprot = list(create_chunks(list_gi_uniprot,1000))
map_uniprot = ThreadPool(10).imap(functools.partial(mapping_ids,source_fmt='ACC',target_fmt='KO_ID'),chunk_uniprot)

for index,ii in enumerate(map_uniprot): 
    with open('D:/eulle/importante/uniprot-ko_id/'+ secrets.token_hex(15) +'.csv', 'w') as file:
        file.write(ii)



In [26]:
uniprot_ko_concat = concat_dfs(os.chdir("D:/eulle/importante/uniprot-ko_id/"),"csv")
uniprot_ko_concat.columns = ["Uniprot_ID","KO"]
uniprot_ko_concat = uniprot_ko_concat.reset_index(drop=True)

uniprot_ko_concat.to_csv("D:/eulle/importante/df_ko_ids_E7.csv",sep="\t",encoding="utf-8",index=False)

In [12]:
basta_output = pd.read_fwf("C:/Users/eulle/Documents/metagenomica/besthit_CoSQG.txt")
besthits = pd.read_csv("C:/Users/eulle/Documents/metagenomica/besthits_pathways-CoSQG-julia.csv",sep="]")
taxmy = taxonomy_metag(basta_output,besthits)
taxmy
#taxmy.to_csv("C:/Users/eulle/Documents/metagenomica/taxonomy_besthits-CoSQG.csv",sep="]",encoding="utf-8",index=False)


# É SÓ PEGAR O UNIQUE DA TABELA TAXONOMY E EXCLUIR OS UNKNOWN -- FEITO
# SABER QUAIS KOs NÀO FORAM MAPEADOS E CONTAR

,ID,qseqid,Proteins,KO,Pathway,id_read,taxonomy
0,17736947,N2D9U:01641:01705,3-oxoadipyl-CoA_thiolase,K07823,Benzoate degradation;Microbial metabolism in d...,N2D9U:01641:01705,Acinetobacter
1,17736947,N2D9U:02039:00908,3-oxoadipyl-CoA_thiolase,K07823,Benzoate degradation;Microbial metabolism in d...,N2D9U:02039:00908,Acinetobacter
2,17736947,N2D9U:03067:01836,3-oxoadipyl-CoA_thiolase,K07823,Benzoate degradation;Microbial metabolism in d...,N2D9U:03067:01836,Acinetobacter
3,17736947,N2D9U:02256:01640,3-oxoadipyl-CoA_thiolase,K07823,Benzoate degradation;Microbial metabolism in d...,N2D9U:02256:01640,Acinetobacter
4,17736947,N2D9U:02304:01185,3-oxoadipyl-CoA_thiolase,K07823,Benzoate degradation;Microbial metabolism in d...,N2D9U:02304:01185,Acinetobacter
...,...,...,...,...,...,...,...
12744,584451878,N2D9U:00071:00654,GTPases_-_Sulfate_adenylate_transferase_subunit,K00956,Purine metabolism;Monobactam biosynthesis;Sele...,N2D9U:00071:00654,Acinetobacter
12745,584451878,N2D9U:01300:01801,GTPases_-_Sulfate_adenylate_transferase_subunit,K00956,Purine metabolism;Monobactam biosynthesis;Sele...,N2D9U:01300:01801,Acinetobacter
12746,597515631,N2D9U:01329:02207,NaN,K05549,Benzoate degradation;Fluorobenzoate degradatio...,NaN,NaN
12747,Q04747,N2D9U:01938:02826,surfactin_synthetase,K15655,Nonribosomal peptide structures;Quorum sensing,N2D9U:01938:02826,Acinetobacter


In [7]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/718417069ead87650b90472464c7565dc8c2cb1c/coffee-flavors.csv')

fig = go.Figure()

fig.add_trace(go.Sunburst(
    ids=df.ids,
    labels=df.labels,
    parents=df.parents,
    domain=dict(column=1),
    maxdepth=2,
    insidetextorientation='radial'
))

fig.update_layout(
    margin = dict(t=10, l=10, r=10, b=10)
)

fig.show()

In [10]:
set(list(df.ids))

{'Acidy-Nippy',
 'Acidy-Piquant',
 'Alliaceous-Garlic',
 'Alliaceous-Onion',
 'Ashy-Burnt',
 'Ashy-Charred',
 'Berry-like-Apricot',
 'Berry-like-Blackberry',
 'Bitter-Harsh',
 'Bitter-Pungent',
 'Bland-Neutral',
 'Bland-Soft',
 'Candy-like-Roasted Almond',
 'Candy-like-Roasted Hazelnut',
 'Carbony-Ashy',
 'Carbony-Smokey',
 'Carmelly-Candy-like',
 'Carmelly-Syrup-like',
 'Chocolate-like-Bakers',
 'Chocolate-like-Dark Chocolate',
 'Chocolatey-Chocolate-like',
 'Chocolatey-Vanilla-like',
 'Citrus-Apple',
 'Citrus-Lemon',
 'Dry Distillation-Carbony',
 'Dry Distillation-Resinous',
 'Dry Distillation-Spicy',
 'Enzymatic-Flowery',
 'Enzymatic-Fruity',
 'Enzymatic-Herby',
 'Floral-Coffee Blossom',
 'Floral-Tea Rose',
 'Flowery-Floral',
 'Flowery-Fragrant',
 'Fragrant-Cardamon Caraway',
 'Fragrant-Coriander Seeds',
 'Fruity-Berry-like',
 'Fruity-Citrus',
 'Harsh-Alkaline',
 'Harsh-Caustic',
 'Herby-Alliaceous',
 'Herby-Leguminous',
 'Leguminous-Cucumber',
 'Leguminous-Garden Peas',
 'Malt-like